In [1]:
import pandas as pd

In [29]:
df = pd.read_csv(r'E:\Personal\Hands-on Machine Learning With Python\train.csv')

In [30]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [5]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [6]:
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
df.hist(bins=50, figsize=(20,15))
plt.show()

<Figure size 2000x1500 with 9 Axes>

In [7]:
df["Embarked"].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [8]:
df["Pclass"].value_counts()/len(df)

3    0.551066
1    0.242424
2    0.206510
Name: Pclass, dtype: float64

In [9]:
df["Sex"].value_counts()/len(df)

male      0.647587
female    0.352413
Name: Sex, dtype: float64

In [31]:
df_label=df["Survived"]

In [32]:
df=df.drop("Survived", axis=1)

In [33]:
df=df.drop("PassengerId", axis=1)

In [34]:
df=df.drop("Name", axis=1)

In [14]:
#df=df.drop(['Ticket', 'Cabin'], axis=1)

In [35]:
df.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,female,35.0,1,0,113803,53.1000,C123,S
4,3,male,35.0,0,0,373450,8.0500,NaN,S


In [36]:
df_num=df.drop(['Sex', 'Embarked', 'Ticket', 'Cabin'], axis=1)

In [37]:
df_cat=df.drop(['Pclass','Age', 'SibSp', 'Parch','Fare'],axis=1)

In [38]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MultiLabelBinarizer

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names=attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values
class CustomLabelBinarizer(BaseEstimator, TransformerMixin):
    def __init__(self, sparse_output=False):
        self.sparse_output=sparse_output
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        enc=MultiLabelBinarizer(sparse_output=self.sparse_output)
        return enc.fit_transform(X)

In [40]:
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import numpy as np
num_attribs=list(df_num)
cat_attribs=['Sex','Ticket','Cabin', 'Embarked']

num_pipeline=Pipeline([
    ('selector', DataFrameSelector(num_attribs)),
    ('imputer', SimpleImputer(strategy="mean")),
    ('std_scaler', StandardScaler())
])

cat_pipeline=Pipeline([
    ('selector', DataFrameSelector(cat_attribs)),
    ('imputer', SimpleImputer(strategy="most_frequent")),
    ('label_binarizer', CustomLabelBinarizer())
])

full_pipeline = FeatureUnion(transformer_list=[
    ('num_pipeline', num_pipeline),
    ('cat_pipeline', cat_pipeline)
])

In [41]:
df_prepared=full_pipeline.fit_transform(df)

In [55]:
from sklearn.linear_model import LogisticRegression
logit_model=LogisticRegression()
logit_model.fit(df_prepared, df_label)

C:\Users\ali.ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [57]:
logit_model_predictions=logit_model.predict(df_prepared)
logit_model_accuracy=logit_model.score(df_prepared, df_label)

In [58]:
logit_model_accuracy

0.8911335578002245

In [62]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(logit_model, df_prepared, df_label, cv=10, scoring='accuracy')

C:\Users\ali.ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ali.ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ali.ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ali.ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ali.ahmed\AppData\Local\Continuum\anaconda3\lib\site-pa

In [67]:
df_test = pd.read_csv(r'E:\Personal\Hands-on Machine Learning With Python\test.csv')

In [70]:
df_test=df_test.drop('PassengerId', axis=1)

In [73]:
df_test_prepared=full_pipeline.transform(df_test)

In [76]:
df2=pd.DataFrame(df_test_prepared)

In [83]:
len(df2)

418

In [79]:
df1=pd.DataFrame(df_prepared)

In [82]:
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,828,829,830,831,832,833,834,835,836,837
0,0.827377,-0.592481,0.432793,-0.473674,-0.502445,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-1.566107,0.638789,0.432793,-0.473674,0.786845,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.827377,-0.284663,-0.474545,-0.473674,-0.488854,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-1.566107,0.407926,0.432793,-0.473674,0.420730,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.827377,0.407926,-0.474545,-0.473674,-0.486337,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [84]:
dummy_array=np.zeros((418,388))

In [85]:
new_df_test_prepared=np.c_[df_test_prepared, dummy_array]

In [87]:
predictions=logit_model.predict(new_df_test_prepared)

In [88]:
dfpredictions=pd.DataFrame(predictions)

In [92]:
dfpredictions.to_excel(r'E:\Personal\Hands-on Machine Learning With Python\predictions.xlsx')